In [66]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
from collections import defaultdict
from graphviz import Digraph


class Edge:
    
    k = None
    def __init__(self, source, dest):
        self.source = source
        self.dest = dest
        self.sequence = source.sequence + dest.sequence[-1]
        self.coverage = 0
        self.coverages = 0
        self.parts = 2


    def extend(self, second_edge):
        self.coverages += second_edge.coverages - second_edge.source.coverage
        self.sequence += second_edge.sequence[Edge.k:]
        self.dest = second_edge.dest
        self.parts += second_edge.parts - 1

        
class Vertex:
    
    def __init__(self, sequence):
        self.sequence = sequence
        self.coverage = 0
        self.ins = []
        self.outs = []
        self.in_degree = 0
        self.out_degree = 0
    
    
class Graph:
    
    def __init__(self, path, k):
        self.reads = SeqIO.parse(path, 'fastq')
        self.k = k
        Edge.k = k
        self.graph_scheme = {}
        self.edges = defaultdict(list)
        self.collapsed_graph = {}
        self.collapsed_edges = defaultdict(list)
        self.edges_cut = {}
    
    def add_read(self, read):
        kmers = [read[i:i + self.k] for i in range(len(read) - self.k + 1)]
        for source, destination in zip(kmers, kmers[1:]):
            if source in self.graph_scheme and destination in self.graph_scheme:
                self.graph_scheme[source][0].coverage += 1
                self.graph_scheme[destination][0].coverage += 1
                if (source, destination) not in self.edges:
                    self.graph_scheme[source][0].out_degree += 1
                    self.graph_scheme[destination][0].in_degree += 1
                    self.graph_scheme[source][1].append(destination)
                    self.graph_scheme[source][0].outs.append(destination)
                    self.graph_scheme[destination][0].ins.append(source)
                    edge = Edge(self.graph_scheme[source][0], self.graph_scheme[destination][0])
                    self.edges[(source, destination)].append(edge)
                continue

            elif source in self.graph_scheme and destination not in self.graph_scheme:
                self.graph_scheme[destination] = (Vertex(destination), [])
                self.graph_scheme[source][0].outs.append(destination)
                self.graph_scheme[destination][0].ins.append(source)
            
            elif source not in self.graph_scheme and destination in self.graph_scheme:
                self.graph_scheme[source] = (Vertex(source), [])
                self.graph_scheme[destination][0].ins.append(source)
                self.graph_scheme[source][0].outs.append(destination)
            
            else:
                self.graph_scheme[source] = (Vertex(source), [])
                self.graph_scheme[destination] = (Vertex(destination), [])
                self.graph_scheme[source][0].outs.append(destination)
                self.graph_scheme[destination][0].ins.append(source)

            self.graph_scheme[source][0].out_degree += 1
            self.graph_scheme[destination][0].in_degree += 1
            self.graph_scheme[source][0].coverage += 1
            self.graph_scheme[destination][0].coverage += 1
            self.graph_scheme[source][1].append(destination)
            self.edges[(source, destination)].append(Edge(self.graph_scheme[source][0], 
                                                          self.graph_scheme[destination][0]))
    
    def compl(self):
        for read in self.reads:
            self.add_read(str(read.seq).upper())
            self.add_read(str(read.reverse_complement().seq).upper())
                
    def cover_edges(self):
        for edge in self.edges.values():
            for e in edge:
                e.coverages = e.source.coverage + e.dest.coverage
                e.coverage = e.coverages / e.parts
                            
    def edge_coverage(self):
        for edge in self.edges.values():
            for e in edge:
                e.coverage = e.coverages / e.parts
             
    def collapse(self):
        collapsable = list(filter(lambda vertex: vertex.in_degree == 1 and vertex.out_degree == 1, (x[0] for x in self.graph_scheme.values())))
        for collapsing in collapsable:
            self.graph_scheme[collapsing.ins[0]][1].remove(collapsing.sequence)
            self.graph_scheme[collapsing.ins[0]][1].append(collapsing.outs[0])
            self.graph_scheme[collapsing.ins[0]][0].outs.remove(collapsing.sequence)
            self.graph_scheme[collapsing.ins[0]][0].outs.append(collapsing.outs[0])
            self.graph_scheme[collapsing.outs[0]][0].ins.remove(collapsing.sequence)
            self.graph_scheme[collapsing.outs[0]][0].ins.append(collapsing.ins[0])
            first_edge = self.edges[(collapsing.ins[0], collapsing.sequence)][0]
            second_edge = self.edges[(collapsing.sequence, collapsing.outs[0])][0]
            first_edge.extend(second_edge)
            self.edges[(collapsing.ins[0], collapsing.outs[0])].append(first_edge)
            del self.graph_scheme[collapsing.sequence]
            del self.edges[(collapsing.sequence, collapsing.outs[0])]
            del self.edges[(collapsing.ins[0], collapsing.sequence)]

    # ratio_treshhold * mean graph coverage = cut treshhold
    def drop_edges(self, ratio_treshhold, len_treshhold, only_tails = True):
        mean_coverage = sum(getattr(v[0], 'coverage') for v in self.graph_scheme.values()) / len(self.graph_scheme)
        cut_cov = ratio_treshhold * mean_coverage
        cut_len = len_treshhold - self.k
        del_list = []
        for e in a.edges: 
            if only_tails == True:
                drop_cond = (a.edges[e][0].dest.out_degree * a.edges[e][0].source.in_degree == 0)
            else:
                drop_cond = True          
            if (a.edges[e][0].coverage < cut_cov) & drop_cond & (a.edges[e][0].parts <= cut_len):
                del_list.append(e)
        self.edges_cut = dict(self.edges.copy())
        for e in del_list:
            self.edges_cut.pop(e)
                 
    def plot(self, filename, cutted):
        dot = Digraph(format = 'png')
        
        if cutted == True:
            gr_edges = self.edges_cut
        else:
            gr_edges = self.edges
        
        for vs, (v, out) in self.graph_scheme.items():
            for dv in out:
                try:
                    vsdv = gr_edges[(vs, dv)]
                    dot.node(vs, label = '')
                    dot.node(dv, label = '')
                    for e in vsdv:
                        dot.edge(vs, dv, label = '(%s, %.2f)' %((self.k + e.parts - 1),(e.coverage)))
                except:
                    pass
        dot.render(filename)
        
        
def main(filepath, k):
    a = Graph(filepath, k)
    a.compl()
    a.cover_edges()
    a.edge_coverage()
    a.collapse()
    a.edge_coverage()
    return a


In [ ]:
a = main('s_6.first1000.fastq', 55)
a.plot('results/1000', cutted = False)

a.drop_edges(ratio_treshhold = 0.3, len_treshhold = 80, only_tails = True)
a.plot('results/1000_tails_0.3', cutted = True)

a.drop_edges(ratio_treshhold = 0.3, len_treshhold = 80, only_tails = False)
a.plot('results/1000_cut_all_0.3', cutted = True)


a = main('s_6.first10000.fastq', 55)
a.plot('results/10000', cutted = False)

a.drop_edges(ratio_treshhold = 0.3, len_treshhold = 80, only_tails = True)
a.plot('results/10000_tails_0.3', cutted = True)

a.drop_edges(ratio_treshhold = 0.3, len_treshhold = 80, only_tails = False)
a.plot('results/10000_cut_all_0.3', cutted = True)

a.drop_edges(ratio_treshhold = 2, len_treshhold = 80, only_tails = True)
a.plot('results/10000_tails_2', cutted = True)

a.drop_edges(ratio_treshhold = 2, len_treshhold = 80, only_tails = False)
a.plot('results/10000_cut_all_2', cutted = True)


a = main('s_6.first100000.fastq', 55)
a.plot('results/100000', cutted = False)

a.drop_edges(ratio_treshhold = 0.3, len_treshhold = 80, only_tails = True)
a.plot('results/100000_tails_0.3', cutted = True)

a.drop_edges(ratio_treshhold = 0.3, len_treshhold = 80, only_tails = False)
a.plot('results/100000_cut_all_0.3', cutted = True)

a.drop_edges(ratio_treshhold = 2, len_treshhold = 80, only_tails = True)
a.plot('results/100000_tails_2', cutted = True)

a.drop_edges(ratio_treshhold = 2, len_treshhold = 80, only_tails = False)
a.plot('results/100000_cut_all_2', cutted = True)
